# **First CNN Model(PetFinder)**

**Description**
Millions of stray animals suffer on the streets or are euthanized in shelters every day around the world. A good picture of a stray animal might increase its chances of getting adopted. But what makes a good picture?
Our mission is to build an ML model which is able to accurately determine a pet photo's appeal and even suggest improvements to give these rescue animals a higher chance of loving homes.

**Data**
9912 images of pet animals labeled with "Pawpularity".
Photo Metadata = (Focus, Eyes, Face, Near, Action, Accessory, Group, Collage, Human, Occlusion, Info, Blur)

To construct this ML, I have learned and refered the notebooks listed below.

Data transformation : https://www.kaggle.com/manabendrarout/transformers-classifier-method-starter-train

Pytorch tutorial : https://pytorch.org/tutorials/beginner/introyt/modelsyt_tutorial.html





In [ ]:
import numpy as np
import pandas as pd
import os
import path
import random
import glob
import cv2
import albumentations
from albumentations.pytorch.transforms import ToTensorV2

import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# 1. Data loading from CSV files

In [ ]:
df_data = pd.read_csv("../input/petfinder-pawpularity-score/train.csv")
df_test = pd.read_csv("../input/petfinder-pawpularity-score/test.csv")


**1.1 Parameters settings**

Many notebooks use this libarary to store all necessary parameters or config for easy editing

In [ ]:
params = {
    'folder_dir': '../input/petfinder-pawpularity-score/',
    'image_dir': '../input/petfinder-pawpularity-score/train/',
    'img_size' : 384,
    'test_img_dir': '../input/petfinder-pawpularity-score/test',
    'device' : 'gpu'
}

1.2 **Augmentation**

Augmentation is used to transform image data into the desired type or shape, normalize it and turn it into a tensor form. At the same time, augmentation is also used to create multiple images out of the same image input by flipping, rotating or mixing up a few different images. This helps us to increase the volume of our data that we can use to train our model, which in the end will help the model predict general data correctly.

In [ ]:
def Transform_data(DIM = params['img_size']):
    return albumentations.Compose(
        [
            albumentations.Resize(DIM, DIM),
            albumentations.Normalize(
                mean = [0.485, 0.456, 0.406],
                std = [0.229, 0.224, 0.225]
            ),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.Rotate(limit = 180, p=0.7),
            albumentations.HueSaturationValue(
                hue_shift_limit=0.2, sat_shift_limit=0.2,
                val_shift_limit=0.2, p=0.5
            ),
            albumentations.RandomBrightnessContrast(
                brightness_limit = (-0.1, 0.1),
                contrast_limit = (-0.1, 0.1), p=0.5
            ),
            ToTensorV2(p=1.0)
        ]
    )


In [ ]:
def Transform_val(DIM = params['img_size']):
    return albumentations.Compose(
        [
            albumentations.Resize(DIM, DIM),
            albumentations.Normalize(
                mean = [0.485, 0.456, 0.406],
                std = [0.229, 0.224, 0.225]
            ),
            ToTensorV2(p=1.0)
        ]
    )

**1.3 Data preparation**

Class created to simplify data preparation. Images are loaded and tranformed inside this class, and the resulting images and labels are returned for further process.

In [ ]:
class PawDataSet():
    def __init__(self,dataset, params, transform = None):
        self.dataset = dataset
        self.image_path = dataset['Id'].apply(lambda x: os.path.join(params['image_dir'],f'{x}.jpg'))
        self.target_label = dataset['Pawpularity']
        self.transform = transform
        self.params = params

    def __len__(self):
        return len(self.image_path)

    def __getitem__(self, idx):
        image_filepath = self.image_path[idx]
        image = cv2.imread(image_filepath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transform is not None:
            image = self.transform(image=image)['image']

        label = torch.tensor(self.target_label[idx]).float()
        return image, label


In [ ]:
class TestDataSet():
    def __init__(self,dataset, params, transform = None):
        self.dataset = dataset
        self.image_path = dataset['Id'].apply(lambda x: os.path.join(params['test_img_dir'],f'{x}.jpg'))
        self.transform = transform
        self.params = params

    def __len__(self):
        return len(self.image_path)

    def __getitem__(self, idx):
        image_filepath = self.image_path[idx]
        image = cv2.imread(image_filepath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transform is not None:
            image = self.transform(image=image)['image']
        return image



In [ ]:
# Load our training data through the class created above.
train_df = PawDataSet(dataset = df_data, params=params, transform = Transform_data())
test_df = TestDataSet(dataset = df_test, params=params, transform = Transform_val())

**1.4 Image visualization**

In [ ]:
def show_image(train_dataset = train_df, inline=4):
    plt.figure(figsize=(20, 10))
    for i in range(inline):
        rand = random.randint(0, len(train_dataset))
        image, label = train_dataset[rand]
        plt.subplot(1, inline, i%inline+1)
        plt.axis('off')
        plt.imshow(image.permute(2, 1, 0))
        plt.title(f'Pawpularity: {label}')

In [ ]:
for i in range(3):
    show_image(inline=4)

**1.5 Data Loading**

Data(image, label) created above is loaded into dataset by using torch.utils.data.DataLoader. At the same time splitted into many small batches.

In [ ]:
train_loader = torch.utils.data.DataLoader(train_df, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_df, batch_size=1, shuffle=True)

# 2. Model building

**2.1 Model**

For my first CNN model, I mostly followed the tutorial in PyTorch documentation. The model includes two convolution layers and three fully-connected layers.

In convolution layers, filters are multiplied through the pixels of every image to get a feature map. Then in fully-connected layers, data from the feature maps are compiled into smaller sizes that were specified.

In between the covolution layers, I have chosen ReLU(Rectified Linear Unit) as my linear function. This fuction drops negative numbered data while keeping the positive data as it is. Lastly, there is a Pooling layer which amplifies the data value while shrinking the size of image.

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.conv3 = nn.Conv2d(32, 64, 3)
        self.fc1 = nn.Linear(64*46*46, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)
        self.drop = nn.Dropout(p=0.2)
        self.count = 1
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.drop(x)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = x.squeeze(1)
        return x   
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features =1 
        for s in size:
            num_features *= s
        return num_features

**Loading model into GPU(cuda)**
At the code below, the model that I created are loaded into the GPU to accelerate the calculation.

In [ ]:
net=Net()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net.to(device)

**Loss and optimizer**

The loss and optimizer that I am going to use to optimize my model are coded below. 
I used criterion to calculate the loss (loss = model output - target "Pawpularity) and pass the loss through the optimizer to change the weight of the filter that will be used in the convolution layer.

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.AdamW(net.parameters(), lr=0.001)

# 3. Train model

Training dataset are used to train the model here.

In [ ]:
for epoch in range(2):
    
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        inputs, label = data
        inputs, label = data[0].to(device), data[1].to(device)
        
        optimizer.zero_grad()
        preds = net(inputs)
        loss = criterion(preds, label)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        print('[%d, %5d] loss: %.3f' %(epoch+1, i+1, running_loss/64))
        running_loss=0.0
        
print('Finished training()'.format(device))

In [ ]:
final_result = []
for i, data in enumerate(test_loader):
    image = data[0]
    image = image.unsqueeze(0)
    image = image.to(device)
    print(image.size())
    preds = net(image).detach().cpu()
    preds = preds.squeeze()
    preds = preds.tolist()
    final_result.append(preds)
    
print(final_result)

for i in final_result:
    print(i)

df_test["Pawpularity"] = final_result
df_test = df_test[['Id','Pawpularity']]
df_test.to_csv("submission.csv", index=False)

In [ ]:
df_test

**Conclusion**

According to the printed loss, we have trained our model to have a mean squared error of about 5-10. 

By constructing this notebook, I have learned a lot on how a CNN Model is built and run.
First, I have learned about the style of coding (Object Oriented Programming) from the notebook that I refered from. Then, through pytorch's tutorials, I have learned how pytorch helps to keep track of the gradient descent and update the parameters of filter. 

For the next notebook.
I would like to explore on how to build a model using pretrained model and construct my model so that I can test it on test data. Besides that, I would like to explore Kfold data and cross validations.